In [1]:
import math
import numpy as np

In [2]:
class Point_projective():
    def __init__(self, X, Y, Z):
        self.X = X
        self.Y = Y
        self.Z = Z

class Curve_parameter():
    def __init__(self, a, b, p):
        self.p = p
        self.a = a
        self.b = b

def print_point(P):
    print("X= ",hex(P.X))
    print("Y= ",hex(P.Y))
    print("Z= ",hex(P.Z))

# Montgomery multiplier parameter

In [3]:
# Montgomery Multiplier parameter

def conv2digit(x, digit_size, s):
    t=x
    T = []
    for i in range (s):
        T.append(0)
    i=0
    while (t > 0):
        T[i] = t & (2**digit_size-1)
        t= t >> digit_size;
        i += 1
    return T

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
    
def set_Mont_parameter(prime, wd_sz):
    s = math.ceil((len(bin(prime))-2)/wd_sz) + 1 # number of words

    fd_sz = wd_sz*s #feild size
    R = 2**wd_sz
    R2 = (2**fd_sz) **2 % prime
    mu_word= (-modinv(prime, R)) % R

    prime_word={}
    for i in range(s):
        prime_word[i] = (prime >> wd_sz*i) % R
    
    return s, fd_sz, R2, mu_word, prime_word

def sum_carry(x):
    S = x & (2**wd_sz - 1)
    C = x >> wd_sz
    return (C,S)

#if mu_word = 1
def Mont_mult(X, Y):
    X_= conv2digit(X, wd_sz, s+1)
    Y_= conv2digit(Y, wd_sz, s+1)
    T = conv2digit(0, wd_sz, s+1)
    for i in range(s):
        (C,S) = sum_carry((T[0] +  (X_[i] * Y_[0])))
        m = S
        C = C + S
        for j in range(1, s):
            (C,S) = sum_carry(T[j] + m*prime_word[j] + X_[i]*Y_[j] + C)
            T[j-1] = S
        T[s-1] = C & (2**wd_sz - 1)
    
    T_total = 0
    for j in range(s):
        T_total += (T[j] << j*wd_sz)
    return T_total

# finite field arithmetic

In [4]:
# finite field arithmetic including modular multiplier, modular inversion

def fp_mult(a, b, p):
    if(mont_flag):
        return Mont_mult(a, b)
    else:
        return a * b % p
    
def fp_inv(a, p):
    return FLT_inv(a, p)

def FLT_inv(a, p):
    window_size = 3
    precompute = {}

    if(mont_flag):
        precompute[0] = Mont_mult(1, R2)
    else:
        precompute[0] = 1
    for i in range(1, 2**window_size):
        precompute[i] = fp_mult(a, precompute[i-1], p)
        
    power = p - 2
    window_no = math.ceil(len(bin(power)[2:])/window_size)
    a_inv = precompute[0]
    for i in range(window_no-1, -1, -1):
        for j in range(window_size):
            a_inv = fp_mult(a_inv, a_inv, p)
        
        idx = (power >> (i*window_size)) & (2**window_size - 1)
        a_inv = fp_mult(a_inv, precompute[idx], p)
    
    return a_inv

# point multiplication and group operations

In [5]:
# Montgomery ladder to perform point multiplication
# point multiplication includes two main group operaitons: Point Addition (PA), and Point Doubling (PD)
# group operations from https://hyperelliptic.org/EFD/g1p/auto-shortw-projective.html#addition-add-1998-cmo-2


def ladder_step(E, P, k):  # point multiplication over curve E and base point P respect to scalar k
    R = []
    #R[0] = O
    R.append(P)
    #R[1] = P
    R.append(PD(E, P))
    
    l = len(bin(k)[2:])
    for i in range(l-2, -1, -1):
        t = (k >> i) & 1
        R[1-t] = PA(E, R[t], R[1-t])
        R[t] = PD(E, R[t])

    return R[0]

# Point Addition
def PA(E, P1, P2):
    Y1Z2 = fp_mult(P1.Y, P2.Z, p)
    X1Z2 = fp_mult(P1.X, P2.Z, p)
    Z1Z2 = fp_mult(P1.Z, P2.Z, p)
    u  = (fp_mult(P2.Y, P1.Z, p) - Y1Z2 ) % p
    uu = fp_mult(u, u, p)
    v = (fp_mult(P2.X, P1.Z, p) - X1Z2) % p
    vv = fp_mult(v, v, p)
    vvv = fp_mult(v, vv, p)
    R = fp_mult(vv, X1Z2, p)
    RpR = (R + R) % p
    A = (fp_mult(uu, Z1Z2, p) - vvv - RpR) % p
    X3 = fp_mult(v, A, p)
    Y3 = (fp_mult(u, (R-A) % p, p) - fp_mult(vvv, Y1Z2, p)) % p
    Z3 = fp_mult(vvv, Z1Z2, p)
    return Point_projective(X3, Y3, Z3)

# Point Doubling
def PD(E_, P1):
    XX = fp_mult(P1.X, P1.X, p)
    ZZ = fp_mult(P1.Z, P1.Z, p)
    XXp3 = (XX + XX + XX) % p
    w = (fp_mult(E_.a, ZZ, p) + XXp3) % p
    s_h = fp_mult(P1.Y, P1.Z, p)
    s = (s_h + s_h) % p
    ss = fp_mult(s, s, p)
    sss = fp_mult(s, ss, p)
    R = fp_mult(P1.Y, s, p)
    RR = fp_mult(R, R, p)
    B = (fp_mult((P1.X + R), (P1.X + R), p) - XX - RR) % p
    BpB = (B + B) % p
    h = (fp_mult(w, w, p) - BpB) % p
    X3 = fp_mult(h, s, p)
    RRpRR = (RR + RR) % p
    Y3 = (fp_mult(w, (B - h) % p, p) - RRpRR) % p
    Z3 = sss
    return Point_projective(X3, Y3, Z3)

# point multiplication test

In [6]:
def test_ECPM(E, scalar, G_aff, Q):
    
    # regular point multiplication without Montgomery multiplier
    if (mont_flag == False): 
        Z = 1
        G = Point_projective(G_aff[0] * Z, G_aff[1] * Z, Z)
        O = Point_projective(1, 1, 0)

        k = fix_MSB(scalar)
        #k = scalar
        Q_proj = ladder_step(E, G, k)
        Z_inv = fp_inv(Q_proj.Z, p)
        Q_aff = [fp_mult(Q_proj.X, Z_inv, p), fp_mult(Q_proj.Y, Z_inv, p)]

        if (Q):    
            if (Q_aff == Q):
                print('test passed!')
            else:
                print('test failed!')
                print('scalar=',hex(k),'\n')
        return Q_aff
    
    # point multiplication with Montgomery multiplier
    else:
        E_mont = Curve_parameter(0, 0, p)
        E_mont.a = Mont_mult(E.a, R2)
        E_mont.b = Mont_mult(E.b, R2)

        Z = 1
        G = Point_projective(G_aff[0] * Z, G_aff[1] * Z, Z)
        G_mont = Point_projective(0, 0, 0)
        G_mont.X = Mont_mult(G.X, R2)
        G_mont.Y = Mont_mult(G.Y, R2)
        G_mont.Z = Mont_mult(G.Z, R2)

        k = fix_MSB(scalar)
        #k = scalar
        Q_proj = ladder_step(E_mont, G_mont, k)
        #print_point(Q_proj)
        Z_inv = fp_inv(Q_proj.Z, p)
        #print('Z_inv=',hex(Z_inv))

        Q_aff_mont = [fp_mult(Q_proj.X, Z_inv, p), fp_mult(Q_proj.Y, Z_inv, p)]
        Q_aff = [fp_mult(Q_aff_mont[0], 1, p), fp_mult(Q_aff_mont[1], 1, p)]

        if (Q):    
            if (Q_aff == Q):
                print('test passed!')
            else:
                print('test failed!')
        return Q_aff

def fix_MSB(scalar):
    scalar_q = scalar + q
    scalar_2q = scalar + 2*q
    
    if ((scalar_q >> 384) == 1):
        return scalar_q
    else:
        return scalar_2q

#   secp384r1 curve parameters
## https://datatracker.ietf.org/doc/html/rfc5114.html#section-2.7   

In [7]:
# curve definition
# y^2 = x^3 + ax + b
a = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffeffffffff0000000000000000fffffffc
b = 0xb3312fa7e23ee7e4988e056be3f82d19181d9c6efe8141120314088f5013875ac656398d8a2ed19d2a85c8edd3ec2aef
p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffeffffffff0000000000000000ffffffff

E = Curve_parameter(a, b, p)

q = 0xffffffffffffffffffffffffffffffffffffffffffffffffc7634d81f4372ddf581a0db248b0a77aecec196accc52973
h = 0x1

# secp384 base point
xp = 0xaa87ca22be8b05378eb1c71ef320ad746e1d3b628ba79b9859f741e082542a385502f25dbf55296c3a545e3872760ab7
yp = 0x3617de4a96262c6f5d9e98bf9292dc29f8f41dbd289a147ce9da3113b5f0b8c00a60b1ce1d7e819d7a431d7c90ea0e5f
G = [xp, yp]

# Point multiplication test without Montgomery multiplier
## test vectors from  https://datatracker.ietf.org/doc/html/rfc5114.html#page-19

In [8]:
dA   = 0xD27335EA71664AF244DD14E9FD1260715DFD8A7965571C48D709EE7A7962A156D706A90CBCB5DF2986F05FEADB9376F1
x_qA = 0x793148F1787634D5DA4C6D9074417D05E057AB62F82054D10EE6B0403D6279547E6A8EA9D1FD77427D016FE27A8B8C66
y_qA = 0xC6C41294331D23E6F480F4FB4CD40504C947392E94F4C3F06B8F398BB29E42368F7A685923DE3B67BACED214A1A1D128

dB   = 0x52D1791FDB4B70F89C0F00D456C2F7023B6125262C36A7DF1F80231121CCE3D39BE52E00C194A4132C4A6C768BCD94D2
x_qB = 0x5CD42AB9C41B5347F74B8D4EFB708B3D5B36DB65915359B44ABC17647B6B9999789D72A84865AE2F223F12B5A1ABC120
y_qB = 0xE171458FEAA939AAA3A8BFAC46B404BD8F6D5B348C0FA4D80CECA16356CA933240BDE8723415A8ECE035B0EDF36755DE

x_Z = 0x5EA1FC4AF7256D2055981B110575E0A8CAE53160137D904C59D926EB1B8456E427AA8A4540884C37DE159A58028ABC0E
y_Z = 0x0CC59E4B046414A81C8A3BDFDCA92526C48769DD8D3127CAA99B3632D1913942DE362EAFAA962379374D9F3F066841CA

In [9]:
mont_flag = False

Q_A = test_ECPM(E, dA, G, [x_qA, y_qA])

Q_B = test_ECPM(E, dB, G, [x_qB, y_qB])

Q_AB = test_ECPM(E, dB, Q_A, [x_Z, y_Z])
Q_BA = test_ECPM(E, dA, Q_B, [x_Z, y_Z])

test passed!
test passed!
test passed!
test passed!


# Point multiplication test with Montgomery multiplier

In [10]:
# Montgomery multiplier settings
wd_sz = 32 # word size
s, fd_sz, R2, mu_word, prime_word = set_Mont_parameter(p, wd_sz)

In [11]:
mont_flag = True

Q_A = test_ECPM(E, dA, G, [x_qA, y_qA])

Q_B = test_ECPM(E, dB, G, [x_qB, y_qB])

Q_AB = test_ECPM(E, dB, Q_A, [x_Z, y_Z])
Q_BA = test_ECPM(E, dA, Q_B, [x_Z, y_Z])

test passed!
test passed!
test passed!
test passed!
